In [ ]:
#code:
#https://github.com/google-deepmind/graphcast.git
# data:
#https://console.cloud.google.com/storage/browser/dm_graphcast;tab=objects?prefix=&forceOnObjectsSortingFiltering=false

In [ ]:
import dataclasses
import xarray
import matplotlib.pyplot as plt
import sys
sys.path.append('graphcast')
from graphcast import graphcast, checkpoint, normalization, autoregressive, casting, data_utils, rollout
import jax
import haiku as hk
import numpy as np
import functools

import pandas as pd

In [ ]:
## all
import dataclasses
import datetime
import functools
import math
import re
from typing import Optional

import cartopy.crs as ccrs
from google.cloud import storage
from graphcast import autoregressive
from graphcast import casting
from graphcast import checkpoint
from graphcast import data_utils
from graphcast import graphcast
from graphcast import normalization
from graphcast import rollout
from graphcast import xarray_jax
from graphcast import xarray_tree
from IPython.display import HTML
import ipywidgets as widgets
import haiku as hk
import jax
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import xarray


In [ ]:
# install ipywidgets
#! pip install ipywidgets

In [ ]:

src_diffs_stddev_by_level = "data/stats/diffs_stddev_by_level.nc"
src_mean_by_level = "data/stats/mean_by_level.nc"
src_stddev_by_level = "data/stats/stddev_by_level.nc"

with open(src_diffs_stddev_by_level, "rb") as f:
    diffs_stddev_by_level = xarray.load_dataset(f).compute()
with open(src_mean_by_level, "rb") as f:
    mean_by_level = xarray.load_dataset(f).compute()
with open(src_stddev_by_level, "rb") as f:
    stddev_by_level = xarray.load_dataset(f).compute()



In [ ]:
src = "data/params/GraphCast_ERA5_1979-2017_Resolution-0.25_PressureLevels-37_Mesh-2to6_PrecipitationInputOutput.npz"
with open(src, "rb",) as f:
    ckpt = checkpoint.load(f, graphcast.CheckPoint)

params = ckpt.params
state = {}

model_config = ckpt.model_config
task_config = ckpt.task_config
print("Model description:/n", ckpt.description, "/n")
print("Model license:/n", ckpt.license, "/n")

In [ ]:
example_batch_src = "data/datasets/source-era5_date-2022-01-01_res-0.25_levels-37_steps-01.nc"
with open(example_batch_src, "rb") as f:
    example_batch = xarray.load_dataset(f).compute()

eval_steps = 1
eval_inputs, eval_targets, eval_forcings = data_utils.extract_inputs_targets_forcings(
    example_batch, target_lead_times=slice("6h", f"{eval_steps*6}h"),
    **dataclasses.asdict(task_config))

In [ ]:
#task_config.target_variables

In [ ]:
def construct_wrapped_graphcast(
    model_config: graphcast.ModelConfig,
    task_config: graphcast.TaskConfig):
  """Constructs and wraps the GraphCast Predictor."""
  # Deeper one-step predictor.
  predictor = graphcast.GraphCast(model_config, task_config)

  # Modify inputs/outputs to `graphcast.GraphCast` to handle conversion to
  # from/to float32 to/from BFloat16.
  predictor = casting.Bfloat16Cast(predictor)

  # Modify inputs/outputs to `casting.Bfloat16Cast` so the casting to/from
  # BFloat16 happens after applying normalization to the inputs/targets.
  predictor = normalization.InputsAndResiduals(
      predictor,
      diffs_stddev_by_level=diffs_stddev_by_level,
      mean_by_level=mean_by_level,
      stddev_by_level=stddev_by_level)

  # Wraps everything so the one-step model can produce trajectories.
  predictor = autoregressive.Predictor(predictor, gradient_checkpointing=True)
  return predictor



def run_forward(model_config, task_config, inputs, targets_template, forcings):
  predictor = construct_wrapped_graphcast(model_config, task_config)
  return predictor(inputs, targets_template=targets_template, forcings=forcings)


def grads_fn(params, state, model_config, task_config, inputs, targets, forcings):
  def _aux(params, state, i, t, f):
    (loss, diagnostics), next_state = loss_fn.apply(
        params, state, jax.random.PRNGKey(0), model_config, task_config,
        i, t, f)
    return loss, (diagnostics, next_state)
  (loss, (diagnostics, next_state)), grads = jax.value_and_grad(
      _aux, has_aux=True)(params, state, inputs, targets, forcings)
  return loss, diagnostics, next_state, grads

# Jax doesn't seem to like passing configs as args through the jit. Passing it
# in via partial (instead of capture by closure) forces jax to invalidate the
# jit cache if you change configs.
def with_configs(fn):
  return functools.partial(
      fn, model_config=model_config, task_config=task_config)

# Always pass params and state, so the usage below are simpler
def with_params(fn):
  return functools.partial(fn, params=params, state=state)

# Our models aren't stateful, so the state is always empty, so just return the
# predictions. This is requiredy by our rollout code, and generally simpler.
def drop_state(fn):
  return lambda **kw: fn(**kw)[0]

# Transform the function with Haiku
run_forward = hk.transform_with_state(run_forward)

init_jitted = jax.jit(with_configs(run_forward.init))

#grads_fn_jitted = with_params(jax.jit(with_configs(grads_fn)))
run_forward_jitted = drop_state(with_params(jax.jit(with_configs(
    run_forward.apply))))


In [ ]:
print("Inputs:  ", eval_inputs.dims.mapping)
print("Targets: ", eval_targets.dims.mapping)
#print("Forcings:", eval_forcings.dims.mapping)

"""predictions = rollout.chunked_prediction(
    run_forward_jitted,
    rng=jax.random.PRNGKey(0),
    inputs=eval_inputs,
    targets_template=eval_targets * np.nan,
    forcings=eval_forcings)
predictions"""
# 10 min auf cpu
# halle lan 51.5, lon 11.9

In [ ]:
print("Predictions: ", predictions.dims.mapping)
print("PREDICIONT: ", predictions)

In [ ]:
# For NetCDF format
predictions.to_netcdf('predicted_dataset.nc')


In [67]:
#read in the predicted dataset
src = "predicted_dataset.nc"
with open(src, "rb",) as f:
    predicted_dataset = xarray.load_dataset(f).compute()

In [ ]:
print("Predictions: ", predicted_dataset.dims.mapping)
print("Predictions: ", predicted_dataset.dims)
print("Predictions: ", predicted_dataset)

# write the full print to a file
sys.stdout = open('output.txt', 'w')
print("Predictions: ", predicted_dataset)

## Pandas analysis

In [ ]:
def xarray_to_dataframe(dataset, variables=None, level=None, time=None, batch=None):
    """
    Convert an xarray dataset to a pandas DataFrame.

    :param dataset: xarray.Dataset
    :param variables: List of variables to include in the DataFrame. If None, all variables are included.
    :param level: Specific level to filter on. If None, includes all levels.
    :param time: Specific time to filter on. If None, includes all times.
    :param batch: Specific batch to filter on. If None, includes all batches.
    :return: pandas.DataFrame
    """
    # Select specific level, time, and batch if provided
    if level is not None:
        dataset = dataset.sel(level=level)
    if time is not None:
        dataset = dataset.sel(time=time)
    if batch is not None:
        dataset = dataset.sel(batch=batch)

    # Select specific variables if provided
    if variables is not None:
        dataset = dataset[variables]

    # Convert to DataFrame
    df = dataset.to_dataframe()

    return df

In [ ]:
predictions_df = xarray_to_dataframe(predicted_dataset, variables=None, level=None, time=None, batch=None)



In [ ]:
predictions_df

In [ ]:
from pandas import IndexSlice as idx

# Assuming 'lat' and 'lon' are part of a MultiIndex
pred_germany_df = predictions_df.loc[idx[:, 47.3:55.1, 5.9:15.2, :, :], :]

ger_xarray = pred_germany_df.to_xarray()


In [ ]:
pred_germany_df

In [ ]:
#predictions_df.to_csv('predictions_df.csv')

## Plots

In [ ]:
# @title Plotting functions

def select(
    data: xarray.Dataset,
    variable: str,
    level: Optional[int] = None,
    max_steps: Optional[int] = None
    ) -> xarray.Dataset:
  data = data[variable]
  if "batch" in data.dims:
    data = data.isel(batch=0)
  if max_steps is not None and "time" in data.sizes and max_steps < data.sizes["time"]:
    data = data.isel(time=range(0, max_steps))
  if level is not None and "level" in data.coords:
    data = data.sel(level=level)
  return data

def scale(
    data: xarray.Dataset,
    center: Optional[float] = None,
    robust: bool = False,
    ) -> tuple[xarray.Dataset, matplotlib.colors.Normalize, str]:
  vmin = np.nanpercentile(data, (2 if robust else 0))
  vmax = np.nanpercentile(data, (98 if robust else 100))
  if center is not None:
    diff = max(vmax - center, center - vmin)
    vmin = center - diff
    vmax = center + diff
  return (data, matplotlib.colors.Normalize(vmin, vmax),
          ("RdBu_r" if center is not None else "viridis"))

def plot_data(
    data: dict[str, xarray.Dataset],
    fig_title: str,
    plot_size: float = 5,
    robust: bool = False,
    cols: int = 4
    ) -> tuple[xarray.Dataset, matplotlib.colors.Normalize, str]:

  first_data = next(iter(data.values()))[0]
  max_steps = first_data.sizes.get("time", 1)
  assert all(max_steps == d.sizes.get("time", 1) for d, _, _ in data.values())

  cols = min(cols, len(data))
  rows = math.ceil(len(data) / cols)
  figure = plt.figure(figsize=(plot_size * 2 * cols,
                               plot_size * rows))
  figure.suptitle(fig_title, fontsize=16)
  figure.subplots_adjust(wspace=0, hspace=0)
  figure.tight_layout()

  images = []
  for i, (title, (plot_data, norm, cmap)) in enumerate(data.items()):
    ax = figure.add_subplot(rows, cols, i+1)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(title)
    im = ax.imshow(
        plot_data.isel(time=0, missing_dims="ignore"), norm=norm,
        origin="lower", cmap=cmap)
    plt.colorbar(
        mappable=im,
        ax=ax,
        orientation="vertical",
        pad=0.02,
        aspect=16,
        shrink=0.75,
        cmap=cmap,
        extend=("both" if robust else "neither"))
    images.append(im)

  def update(frame):
    if "time" in first_data.dims:
      td = datetime.timedelta(microseconds=first_data["time"][frame].item() / 1000)
      figure.suptitle(f"{fig_title}, {td}", fontsize=16)
    else:
      figure.suptitle(fig_title, fontsize=16)
    for im, (plot_data, norm, cmap) in zip(images, data.values()):
      im.set_data(plot_data.isel(time=frame, missing_dims="ignore"))

  ani = animation.FuncAnimation(
      fig=figure, func=update, frames=max_steps, interval=250)
  plt.close(figure.number)
  return HTML(ani.to_jshtml())

In [68]:
predicted_dataset

<xarray.Dataset>
Dimensions:                  (lon: 1440, lat: 721, level: 37, time: 1, batch: 1)
Coordinates:
  * lon                      (lon) float32 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * lat                      (lat) float32 -90.0 -89.75 -89.5 ... 89.75 90.0
  * level                    (level) int32 1 2 3 5 7 10 ... 900 925 950 975 1000
  * time                     (time) timedelta64[ns] 06:00:00
Dimensions without coordinates: batch
Data variables:
    10m_u_component_of_wind  (time, batch, lat, lon) float32 -0.2302 ... -1.041
    10m_v_component_of_wind  (time, batch, lat, lon) float32 -1.16 ... 0.4418
    2m_temperature           (time, batch, lat, lon) float32 248.7 ... 247.6
    mean_sea_level_pressure  (time, batch, lat, lon) float32 9.977e+04 ... 1....
    total_precipitation_6hr  (time, batch, lat, lon) float32 0.00011 ... 1.93...
    geopotential             (time, batch, level, lat, lon) float32 4.93e+05 ...
    specific_humidity        (time, batch, level, lat, lon) float32 3.905e-06...
    temperature              (time, batch, level, lat, lon) float32 284.4 ......
    u_component_of_wind      (time, batch, level, lat, lon) float32 0.7964 .....
    v_component_of_wind      (time, batch, level, lat, lon) float32 -0.07711 ...
    vertical_velocity        (time, batch, level, lat, lon) float32 -0.000369...

In [ ]:
# @title Choose predictions to plot


# Set these variables manually as needed
plot_pred_variable = "2m_temperature"  # example value, replace with your choice
plot_pred_level = 500  # example value, replace with your choice
plot_pred_robust = True  # set to True or False
plot_pred_max_steps = predicted_dataset.dims['']  # or any integer value within the range

# Information message
print("Run the next cell to plot the predictions. Rerunning this cell clears your selection.")


In [ ]:
print(ger_xarray.dims)

In [ ]:
# Example of selecting data with multi-level indexing
selected_data = predicted_dataset.isel(lon=0, lat=0, level=0, time=0, batch=0)
precipitation_data = select(data=selected_data, variable="correct_variable_name")


In [ ]:
ger_total_precipitation_6hr = ger_xarray.total_precipitation_6hr


In [ ]:
precipitation_data = select(data=ger_xarray, variable="total_precipitation")


In [ ]:
# plot the predictions
plot_data(
    {f"Predictions (lead time {i*6}h)": select(predicted_dataset, "precipitation", max_steps=i+1)
     for i in range(0, eval_steps)},
    fig_title="Predictions",
    plot_size=5,
    robust=True,
    cols=4)